In [1]:
import energy_consumption as ec
import json
import math

## Import data

In [2]:
with open("../../data_acquisition/test_data.json", "r") as file:
    complete_road_data = json.load(file)
with open("static_data.json", "r") as file:
    static_data = json.load(file)
with open("vehicle_data.json", "r") as file:
    vehicle_data = json.load(file)
with open("battery_data.json", "r") as file:
    battery_data = json.load(file)

In [3]:
OCV = battery_data["OCV"]
capacity = battery_data["Capacity"]
R_int = battery_data["R_internal"]
motor_eff = vehicle_data["motor_eff"]


In [6]:
for path, sections in complete_road_data.items():
    for section, data in sections.items():
        data["velocity"] = 8.9408
        data["acceleration"] = 0
        tract_power = ec.physical_model(vehicle_data, static_data, data)
        batt_power = ec.battery_power_model(tract_power, motor_eff)
        current = ec.discharge_current(OCV, R_int, batt_power)
        c_rate = ec.find_crate(ec.discharge_current(OCV, R_int, batt_power), capacity)
        energy = ec.get_edge_consumption(batt_power, data)

        print(energy)

1804.7811826048185
4817.976050807139
1223.3301412815836
2609.9731642615693
4948.44753038235
13996.980522735086
2716.054405053969
1858.0893571548916
1470.562209216988
2075.2440694383604
2659.4731564551116
2678.6214298592913
2324.936224065771
2636.953576144407
7618.340269223793
254.5741655183685
1863.9033320674077
0.0
26599.34044962044
15343.73259744019
27242.64280857886
13636.15384189879
0.0
0.0
6849.8033801296515
8652.166234055398
1617.1808903685728
3.6621848760273
419.61872023265846
1136.4772613972063
5569.6821517194685
1542.5248998419422
3372.6123072020237
1085.2668748171059
1411.3312471829233
970.3996209828385
513.3293431350517
1121.1548262706267
820.3754260086628
532.4808596960258
535.5506717982971
1311.5069428003294
2547.9422343922915
1487.5109131008278
4828.670859788735
6557.465250544755
1043.485069553281
249.96826188501248
0.0
0.0
0.0
0.0
7487.206523671512
0.0
2584.3472692878768
4235.619944516299
10127.899008115424
16209.729319467471


/Users/aronfinkelstein/Documents/GitHub/EAV-VRP/src/vehicle_models/energy_consumption.py:32: RuntimeWarning: invalid value encountered in scalar power
  I_l = (OCV - (OCV**2 -4*OCV*P_batt)**0.5)/(2*R_i)


In [5]:

def discharge_current(battery_data: dict, P_batt: float)-> float:
    '''
    Find discharge currrent
    '''
    OCV = battery_data["OCV"]
    R_i = battery_data["R_internal"]   
    if P_batt > (OCV**2) / (4 * R_i):
        raise ValueError("P_batt is too high, causing imaginary current.")
    
    I_l = (OCV - math.sqrt(OCV**2 - 4 * R_i * P_batt)) / (2 * R_i)
    
    return I_l

c_rate = ec.find_crate()

print(discharge_current(battery_data, 4193.659401671112))

TypeError: find_crate() missing 2 required positional arguments: 'current' and 'capacity'